# Install Required Libraries

In [ ]:
!pip install tensorflow-recommenders
!pip install pprintpp


# Import Libraries and Define File Paths

In [ ]:
from typing import Dict, Text
import pprintpp
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

file_paths = {
    "interaction_data": "/content/RAW_interactions.csv",
    "recipe_data": "/content/RAW_recipes.csv",
    "interaction_train": "/content/interactions_train.csv",
    "interaction_test": "/content/interactions_test.csv"
}

# Dictionary to store the DataFrames
data_frames = {}


# Read CSV Files in Chunks and Identify Problematic Rows

In [ ]:
def read_csv_in_chunks(file_path, chunk_size=1000):
    try:
        for chunk in pd.read_csv(file_path, chunksize=chunk_size):
            print(chunk.head())  # Print to verify chunks are read successfully
    except pd.errors.ParserError as e:
        print(f"Error parsing CSV file {file_path}: {e}")
    except Exception as e:
        print(f"An error occurred with file {file_path}: {e}")

def identify_problematic_rows(file_path, start_row, end_row, chunk_size=10):
    for row in range(start_row, end_row, chunk_size):
        try:
            specific_rows = pd.read_csv(file_path, skiprows=range(1, row), nrows=chunk_size)
            print(f"Rows {row} to {row + chunk_size - 1} read successfully")
            print(specific_rows)
        except pd.errors.ParserError as e:
            print(f"Error parsing CSV file at rows {row} to {row + chunk_size - 1}: {e}")
        except Exception as e:
            print(f"An error occurred at rows {row} to {row + chunk_size - 1}: {e}")

def read_single_rows(file_path, start_row, end_row):
    for row in range(start_row, end_row):
        try:
            specific_row = pd.read_csv(file_path, skiprows=range(1, row), nrows=1)
            print(f"Row {row} read successfully")
            print(specific_row)
        except pd.errors.ParserError as e:
            print(f"Error parsing CSV file at row {row}: {e}")
        except Exception as e:
            print(f"An error occurred at row {row}: {e}")

print("Reading interaction_data in chunks to identify problematic rows")
read_csv_in_chunks(file_paths["interaction_data"])

problematic_start = 166900
problematic_end = 167200

print("Narrowing down the problematic range")
identify_problematic_rows(file_paths["interaction_data"], problematic_start, problematic_end)

problematic_chunk_start = 167070
problematic_chunk_end = 167080

print("Reading single rows in the problematic range")
read_single_rows(file_paths["interaction_data"], problematic_chunk_start, problematic_chunk_end)


Streaming output truncated to the last 5000 lines.
                                                   review  
808000  I just have to try this because you write the ...  
808001  Giving this four stars because it wasn't reall...  
808002  Delicious, easy to make cake starting with a c...  
808003  I needed something quick to make for a bake sa...  
808004  Very good fresh pasta dish. Made this and used...  
           user_id  recipe_id        date  rating  \
809000  2000569319      28148  2018-05-08       5   
809001      177582      28148  2018-06-01       0   
809002  2002259036      28148  2018-08-27       0   
809003  2002328086      28148  2018-12-19       4   
809004       38745      41843  2006-05-14       5   

                                                   review  
809000  I've been making these for years and just real...  
809001                                        Delicious!!  
809002  Yey !! I love the mango salsa with this recipe...  
809003  These were good but I 

# Load CSV Files with Robust Error Handling

In [ ]:
try:
    interaction_data = pd.read_csv(file_paths["interaction_data"])
    print("interaction_data loaded successfully.")
    data_frames["interaction_data"] = interaction_data
except pd.errors.ParserError as e:
    print(f"Error parsing interaction_data: {e}")
except Exception as e:
    print(f"An error occurred with interaction_data: {e}")

try:
    recipe_data = pd.read_csv(file_paths["recipe_data"])
    print("recipe_data loaded successfully.")
    data_frames["recipe_data"] = recipe_data
except pd.errors.ParserError as e:
    print(f"Error parsing recipe_data: {e}")
except Exception as e:
    print(f"An error occurred with recipe_data: {e}")

try:
    interaction_train = pd.read_csv(file_paths["interaction_train"])
    print("interaction_train loaded successfully.")
    data_frames["interaction_train"] = interaction_train
except pd.errors.ParserError as e:
    print(f"Error parsing interaction_train: {e}")
except Exception as e:
    print(f"An error occurred with interaction_train: {e}")

try:
    interaction_test = pd.read_csv(file_paths["interaction_test"])
    print("interaction_test loaded successfully.")
    data_frames["interaction_test"] = interaction_test
except pd.errors.ParserError as e:
    print(f"Error parsing interaction_test: {e}")
except Exception as e:
    print(f"An error occurred with interaction_test: {e}")


interaction_data loaded successfully.
recipe_data loaded successfully.
interaction_train loaded successfully.
interaction_test loaded successfully.


# Define the RankingModel Class

In [ ]:
class RankingModel(tf.keras.Model):
    def __init__(self, uniqueUserIds, uniqueFoodIds):
        super().__init__()
        embedding_dimension = 32

        self.user_embeddings = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.StringLookup(
                vocabulary=uniqueUserIds, mask_token=None),
            tf.keras.layers.Embedding(len(uniqueUserIds) + 1, embedding_dimension)
        ])

        self.product_embeddings = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.StringLookup(
                vocabulary=uniqueFoodIds, mask_token=None),
            tf.keras.layers.Embedding(len(uniqueFoodIds) + 1, embedding_dimension)
        ])

        self.ratings = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation="relu"),
            tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dense(1)
        ])

    def call(self, userId, foodId):
        user_embeddings = self.user_embeddings(userId)
        food_embeddings = self.product_embeddings(foodId)
        return self.ratings(tf.concat([user_embeddings, food_embeddings], axis=1))


# Convert Data to Appropriate Types and Prepare TensorFlow Datasets

In [ ]:
interaction_train["user_id"] = interaction_train["user_id"].astype(str)
interaction_train["recipe_id"] = interaction_train["recipe_id"].astype(str)
interaction_train["rating"] = interaction_train["rating"].astype(float)

interaction_test["user_id"] = interaction_test["user_id"].astype(str)
interaction_test["recipe_id"] = interaction_test["recipe_id"].astype(str)
interaction_test["rating"] = interaction_test["rating"].astype(float)

uniqueUserIds = interaction_train.user_id.unique()
uniqueFoodIds = interaction_train.recipe_id.unique()

train_data = tf.data.Dataset.from_tensor_slices(
    {
        "userID": tf.constant(interaction_train.user_id.values),
        "foodID": tf.constant(interaction_train.recipe_id.values),
        "rating": tf.constant(interaction_train.rating.values, dtype=tf.float32)
    }
)

test_data = tf.data.Dataset.from_tensor_slices(
    {
        "userID": tf.constant(interaction_test.user_id.values),
        "foodID": tf.constant(interaction_test.recipe_id.values),
        "rating": tf.constant(interaction_test.rating.values, dtype=tf.float32)
    }
)

for data in train_data.take(1):
    print(data)


{'userID': <tf.Tensor: shape=(), dtype=string, numpy=b'2046'>, 'foodID': <tf.Tensor: shape=(), dtype=string, numpy=b'4684'>, 'rating': <tf.Tensor: shape=(), dtype=float32, numpy=5.0>}


# Define the FoodModel Class and Compile the Model

In [ ]:
class FoodModel(tfrs.Model):
    def __init__(self, uniqueUserIds, uniqueFoodIds):
        super().__init__()
        self.ranking_model = RankingModel(uniqueUserIds, uniqueFoodIds)
        self.task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def call(self, features):
        user_embeddings = self.ranking_model.user_embeddings(features["userID"])
        product_embeddings = self.ranking_model.product_embeddings(features["foodID"])
        return self.ranking_model.ratings(tf.concat([user_embeddings, product_embeddings], axis=1))

    def compute_loss(self, features, training=False):
        labels = features.pop("rating")
        predictions = self(features)
        return self.task(labels=labels, predictions=predictions)

tf.random.set_seed(42)

train_data = train_data.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

model = FoodModel(uniqueUserIds, uniqueFoodIds)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.001))

cached_train = train_data.batch(8192).cache()
cached_test = test_data.batch(4096).cache()

model.fit(cached_train, epochs=25)
model.evaluate(cached_test, return_dict=True)


Epoch 1/25
86/86 [==============================] - 15s 151ms/step - root_mean_squared_error: 4.5334 - loss: 20.5296 - regularization_loss: 0.0000e+00 - total_loss: 20.5296
Epoch 2/25
86/86 [==============================] - 8s 87ms/step - root_mean_squared_error: 4.2615 - loss: 18.1350 - regularization_loss: 0.0000e+00 - total_loss: 18.1350
Epoch 3/25
86/86 [==============================] - 4s 47ms/step - root_mean_squared_error: 3.9223 - loss: 15.3566 - regularization_loss: 0.0000e+00 - total_loss: 15.3566
Epoch 4/25
86/86 [==============================] - 4s 52ms/step - root_mean_squared_error: 3.5221 - loss: 12.3782 - regularization_loss: 0.0000e+00 - total_loss: 12.3782
Epoch 5/25
86/86 [==============================] - 10s 120ms/step - root_mean_squared_error: 3.0917 - loss: 9.5353 - regularization_loss: 0.0000e+00 - total_loss: 9.5353
Epoch 6/25
86/86 [==============================] - 7s 77ms/step - root_mean_squared_error: 2.6610 - loss: 7.0620 - regularization_loss: 0.0000

{'root_mean_squared_error': 1.37497079372406,
 'loss': 2.114839553833008,
 'regularization_loss': 0,
 'total_loss': 2.114839553833008}

# Make Predictions and Print Recommendations

In [ ]:
user_rand = uniqueUserIds[200]  # Assuming user_rand is a specific user ID from your dataset
test_rating = {}
for m in test_data.take(10):
    food_id_str = str(m["foodID"].numpy()[0])  # Convert to string before using
    predictions = model({"userID": tf.convert_to_tensor([user_rand]), "foodID": tf.convert_to_tensor([food_id_str])})
    test_rating[food_id_str] = predictions.numpy()[0][0]

print("Predicted ratings:")
print(test_rating)

for user_rand in uniqueUserIds[:10]:  # Iterate through first 10 users for example
    print(f"Top 3 recommended products for User {user_rand}: ")
    test_rating = {}

    for m in test_data.take(10):  # Assuming you want to take 10 recommendations for each user
        food_id_str = str(m["foodID"].numpy()[0])  # Convert to string
        predictions = model({"userID": tf.convert_to_tensor([user_rand]), "foodID": tf.convert_to_tensor([food_id_str])})
        test_rating[food_id_str] = predictions.numpy()[0][0]

    recommended_recipes = sorted(test_rating, key=test_rating.get, reverse=True)[:3]

    if len(recommended_recipes) == 0:
        print("No recommendations found.")
    else:
        for idx, food_id in enumerate(recommended_recipes, start=1):
            recipe_names = recipe_data.loc[recipe_data['id'] == int(food_id)]['name'].values
            if len(recipe_names) > 0:
                print(f"{idx}. {recipe_names[0]}")


Predicted ratings:
{'52': 4.628978, '49': 4.653368, '50': 4.607419, '56': 4.607419}
Top 3 recommended products for User 2046: 
1. chicken breasts lombardi
2. cafe cappuccino
3. biscotti di prato
Top 3 recommended products for User 1773: 
1. chicken breasts lombardi
2. cafe cappuccino
3. biscotti di prato
Top 3 recommended products for User 2312: 
1. chicken breasts lombardi
2. cafe cappuccino
3. biscotti di prato
Top 3 recommended products for User 2625: 
1. chicken breasts lombardi
2. cafe cappuccino
3. biscotti di prato
Top 3 recommended products for User 2999: 
1. chicken breasts lombardi
2. cafe cappuccino
3. biscotti di prato
Top 3 recommended products for User 2178: 
1. chicken breasts lombardi
2. cafe cappuccino
3. biscotti di prato
Top 3 recommended products for User 3794: 
1. chicken breasts lombardi
2. cafe cappuccino
3. biscotti di prato
Top 3 recommended products for User 2695: 
1. chicken breasts lombardi
2. cafe cappuccino
3. biscotti di prato
Top 3 recommended products f